In [159]:
import pandas as pd


data = pd.read_csv('taxi.csv')
data[['pickup_datetime', 'dropoff_datetime']] = data[['pickup_datetime', 'dropoff_datetime']].apply(pd.to_datetime)

In [161]:
data['day'] = data['pickup_datetime'].dt.day
data['month'] = data['pickup_datetime'].dt.month
data['year'] = data['pickup_datetime'].dt.year
#data

In [162]:
### как я рассчитываю день недели https://lifehacker.ru/kakoj-den-nedeli/
### столетия учитывать излишне
def check_leap_year(year: int) -> bool:
    return (year % 4 == 0) and (year % 100 != 0 or year % 400 == 0)


def get_code_of_year(year: int) -> int:
    last_two_digits = int(year % 100)
    return (6 + last_two_digits + int(last_two_digits / 4)) % 7


def get_code_of_month(month: int, year: int) -> int:
    assert month in range(1, 12 + 1), f'Номер месяца в году может быть от 1 до 12, а на входе месяц {month}'
    months_codes = {
        1: 0 if check_leap_year(year) else 1,
        2: 3 if check_leap_year(year) else 4,
        3: 4, 
        4: 0,
        5: 2,
        6:5,
        7: 0,
        8: 3,
        9: 6,
        10: 1,
        11: 4,
        12: 6
    }
    return months_codes[month]


def get_day_of_week(day: int, month: int, year: int) -> int:
    assert day in range(1, 31 + 1), f'Номер дня в месяце должен быть от 1 до 31 включительно, а на входе день {day}'
    days_of_week = {
        0: 6,
        1: 7,
        2: 1,
        3: 2,
        4: 3,
        5: 4,
        6: 5
    }
    return days_of_week[(day + get_code_of_month(month, year) + get_code_of_year(year)) % 7]

In [163]:
data['day_of_week'] = data.apply(lambda x: get_day_of_week(x['day'], x['month'], x['year']), axis=1)
#data

In [164]:
## выходные - 6 и 7 день
filtered_data = data[data.apply(lambda x: x['day_of_week'] in range(6, 7 + 1), axis=1)]
#filtered_data

In [173]:
grouped_by_months = data.groupby('month')
grouped_by_months.count()['id']

month
1    15899
2    16094
3    17650
4    17257
5    16930
6    16170
Name: id, dtype: int64